In [3]:
/**
 * Solve a quadratic function with constraints (no equality constraint) using the primal active set algorithm.
 *
 * @author Haksun Li
 */

%use s2
import dev.nm.solver.multivariate.constrained.convex.sdp.socp.qp.solver.activeset.QPPrimalActiveSetMinimizer.Solution

// construct a QP problem
fun problem(): QPProblem {
    // construct a quadratic function in this form: 1/2 x'Hx + p'x
    // a semi-definite matrix of quadratic coefficients. (x1 - x3)^2 + (x2 - x4)^2
    val H1: Matrix = DenseMatrix(arrayOf(
        doubleArrayOf(1.0, 0.0, -1.0, 0.0), 
        doubleArrayOf(0.0, 1.0, 0.0, -1.0), 
        doubleArrayOf(-1.0, 0.0, 1.0, 0.0), 
        doubleArrayOf(0.0, -1.0, 0.0, 1.0)))
    // a positive-definite matrix of quadratic coefficients obtained by adding x1^2 + x2^2 + x3^2 + x4^2 to the above
    val H2: Matrix = H1.add(H1.ONE())
    // the linear coefficients of the quadratic function
    val p: Vector = DenseVector(arrayOf(0.0, 0.0, 0.0, 0.0))
    // the quadratic function to solve
    val f: QuadraticFunction = QuadraticFunction(H2, p)

    // construct a greater-than constraint
    // the matrix of coefficients on the LHS of the the greater-than constraint inequality
    val A: Matrix = DenseMatrix(arrayOf(
        doubleArrayOf(1.0, 0.0, 0.0, 0.0), 
        doubleArrayOf(0.0, 1.0, 0.0, 0.0), 
        doubleArrayOf(-1.0, -2.0, 0.0, 0.0), 
        doubleArrayOf(0.0, 0.0, 0.0, 1.0),
        doubleArrayOf(0.0, 0.0, 1.0, 1.0), 
        doubleArrayOf(0.0, 0.0, -1.0, -2.0)))
    // the vector of values on the RHS of the greater-than constraint inequality
    val b: Vector = DenseVector(arrayOf(0.0, 0.0, -2.0, 2.0, 3.0, -6.0))
    // a greater-than constraint, Ax >= b
    val greater: LinearGreaterThanConstraints = LinearGreaterThanConstraints(A, b)

    // solve a quadratic programming problem using the primal active set algorithm
    val problem = QPProblem(f, null, greater, null)
    return problem
}

// construct an QP solver
val solver = QPPrimalActiveSetMinimizer(
    1e-8, // precision
    100) // max number of iterations

// solve the QP problem
val p = problem()
val soln: Solution = solver.solve(p)

// An initial guesses for a solution. Note that the initial guess must satisfy the constraints!
val x0: Vector = DenseVector(2.0, 2.0, 3.0, 3.0)
val qpSoln: QPSolution = soln.search(x0)

// the solution
val minimizer = qpSoln.minimizer()
val fx = p.f().evaluate(minimizer);
println("minimizer: $minimizer, fx = $fx")

minimizer: [0.500000, 1.000000, 1.000000, 2.000000] , fx = 3.75
